In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
from urllib.request import urlretrieve

now = datetime.now().strftime('%Y-%m-%d_%H:%M:%S')

print(f"Notebook execution: {now}")


Notebook execution: 2025-10-02_23:07:38


### **Check url logic**
_____

In [2]:
url = ("https://storage.googleapis.com/nozzle-csv-exports/testing-data/order_items_2_.csv")

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
file_name = f"order_items_{timestamp}.csv"
output_path = "data"
output_path = os.path.join(output_path, file_name)

print(f"data filename: {file_name}")
print(f"output path: {output_path}")

data filename: order_items_20251002_230740.csv
output path: data/order_items_20251002_230740.csv


In [3]:
import requests
from typing import Optional

def check_request(url: str, timeout: Optional[int] = 15) -> Optional[bytes]:
    """This function do the request to the
        url
        
        Args:
            - url
            - timeout
    """
    # Try/catch to check url
    try:
        response = requests.get(url, timeout=timeout)
        response.raise_for_status()  # Check for error codes (classic 404 lol)
        return response.content
    
    except requests.Timeout:
        print(f"response took more than {timeout} seconds")
        return None

    except requests.HTTPError as e:
        print(f"HTTP Error {e.response.status_code}: {e}")
        return None
    
    except requests.RequestException as e:
        print(f"Download failed: {e}")
        return None

data = check_request("https://people.sc.fsu.edu/~jburkardt/data/csv/airtravel.csv")
#Test url "https://people.sc.fsu.edu/~jburkardt/data/csv/airtravel.csv"

In [4]:
data # bytes

b'"Month", "1958", "1959", "1960"\n"JAN",  340,  360,  417\n"FEB",  318,  342,  391\n"MAR",  362,  406,  419\n"APR",  348,  396,  461\n"MAY",  363,  420,  472\n"JUN",  435,  472,  535\n"JUL",  491,  548,  622\n"AUG",  505,  559,  606\n"SEP",  404,  463,  508\n"OCT",  359,  407,  461\n"NOV",  310,  362,  390\n"DEC",  337,  405,  432\n\n'

### **Download the file**
_______

In [5]:
from pathlib import Path

def download_file(url_data):
    """
        Download the data file to a .csv file,
        creates the /data folder and saves 
        the downloaded data in the /data folder
        with the name: 
        order_items_<timestamp>.csv

        Arg:
    """
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"order_items_{timestamp}.csv"

    data_folder = Path("../data")
    data_folder.mkdir(parents=True, exist_ok=True)

    output_file = data_folder / filename

    # Save to file
    with output_file.open('wb') as f:
        f.write(url_data)

download_file(data)

### **Clean data**
______

In [11]:
data = pd.read_csv("../data/order_items_data_2_.csv", sep=",")
discarded_rows = []
print(data.dtypes)
print(f"shape: {data.shape}")
print(data.sample(2))

order_id                   object
purchased_at               object
purchased_date             object
purchased_month_ended      object
order_item_id              object
sku                        object
product_title              object
product_name_full          object
currency                   object
item_price                 object
item_tax                   object
shipping_price             object
shipping_tax               object
gift_wrap_price            object
gift_wrap_tax              object
item_promo_discount        object
shipment_promo_discount    object
ship_service_level         object
dtype: object
shape: (5430317, 18)
                                     order_id                purchased_at  \
1295482  a1716ffe-9ce8-4c8b-8d55-cc238ed07839  2025-02-22T00:13:16.184576   
4180656  74d6bd94-769a-42f1-ad9c-1a9dbffcc504  2025-01-17T17:01:13.719529   

        purchased_date purchased_month_ended   order_item_id           sku  \
1295482     2025-02-22            2025-02-2

#### **Remove duplicates**
_____

In [12]:
duplicates = data[data.duplicated(keep='first')]
data_without_duplicates = data.drop_duplicates(keep="first")
discarded_rows.append(duplicates)
print(f"original shape: {data.shape}")
print(f"duplicates shape: {duplicates.shape}")
print(f"without duplicates shape: {data_without_duplicates.shape}")
print(f"Discarded log elements: {len(pd.concat(discarded_rows, ignore_index=True))}")

original shape: (5430317, 18)
duplicates shape: (1104476, 18)
without duplicates shape: (4325841, 18)
Discarded log elements: 1104476


In [13]:
discarded_rows

[                                     order_id                purchased_at  \
 19                                        NaN                         NaN   
 33                                        NaN                         NaN   
 35                                        NaN                         NaN   
 52                                        NaN                         NaN   
 59                                        NaN                         NaN   
 ...                                       ...                         ...   
 5430301  5f76c263-c79a-4e7b-9458-e3241db790fa  2025-04-10T22:27:43.648907   
 5430303  3a6dc8b0-c206-4ae8-946e-1af9a376f464  2025-01-01T05:30:20.944186   
 5430304                                   NaN                         NaN   
 5430306  6428c2e1-d8de-4e2e-8e84-f9b1bb56fd15  2025-02-14T17:28:15.012963   
 5430309  68a81cd0-bb98-4c16-8d45-493bfb0e4e7f  2025-04-29T03:29:47.292819   
 
         purchased_date purchased_month_ended   order_item_id 

#### **Drop empty rows**
_____

In [14]:
empty_rows = data[data.isna().all(axis=1)]
discarded_rows.append(empty_rows)
data_without_duplicates_and_null = data_without_duplicates.dropna(how='all')
print(f"without duplicates and nulls shape: {data_without_duplicates_and_null.shape}")
print(f"Discarded log elements: {len(pd.concat(discarded_rows, ignore_index=True))}")

without duplicates and nulls shape: (4325840, 18)
Discarded log elements: 1630086


#### **Log discarded rows**
_____

In [15]:
# discarded_rows = []
if not discarded_rows:
    discarded_rows_df = pd.DataFrame()  # Return empty if nothing discarded
    
discarded_rows_df = pd.concat(discarded_rows, ignore_index=True) 
print(f"Discarded log elements: {discarded_rows_df.shape}")

Discarded log elements: (1630086, 18)


#### **Save discarded rows**
_____

In [16]:
discarded_rows_df.to_csv("../data/discarded_rows.csv", index=False)

### **Reports**
____

#### **Processing stats report**
_____

In [17]:
print(f"total_rows: {len(data)}")
print(f"empty_rows: {len(empty_rows)}")
print(f"invalid_rows_discarded: {len(discarded_rows_df)}")
print(f"total_duplicate_rows: {len(duplicates)}")
print(f"total_usable_rows: {len(data_without_duplicates_and_null)}")



total_rows: 5430317
empty_rows: 525610
invalid_rows_discarded: 1630086
total_duplicate_rows: 1104476
total_usable_rows: 4325840


In [19]:
import json

stats_dict = {
        "total_rows": len(data),
        "total_empty_rows_removed": len(empty_rows),
        "total_invalid_rows_discarded": len(discarded_rows_df),
        "total_duplicate_rows_removed": len(duplicates),
        "total_usable_rows": len(data_without_duplicates_and_null)
    }

output_path_json = os.path.join("../data", "processing_stats.json")
with open(output_path_json, "w") as f:  # open and write the file
            json.dump(stats_dict, f, indent=4)

#### **Monthly metrics report**
_______